In [22]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [32]:
file1 = 'transactions.csv'
trans_1 = pd.read_csv(file1)

In [33]:
file2 = 'transactions_v2.csv'
trans_2 = pd.read_csv(file2)

In [34]:
trans = trans_1.append(trans_2).reset_index(drop=True)

In [35]:
trans.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [36]:
trans = trans.sort_values(['msno','transaction_date','membership_expire_date'])

In [38]:
#create an array of all months (from Jan 2016 to Apr 2017).
targets = [201601,201602,201603,201604,201605,201606,201607,201608,201609,201610,201611,201612,201701,201702,201703,201704]

In [39]:
#drop duplicate transaction records from transaction logs table
trans = trans.drop_duplicates()
trans.duplicated().sum()

0

In [40]:
current_msno = ''
#create arrays that store final msnos and their value of is_churn
msnos = [[] for i in range(len(targets))]
is_churns = [[] for i in range(len(targets))]

is_cancels = []
transaction_dates = []
membership_expire_dates = []

total_rows = len(trans['msno'])
total_rows

22975416

In [41]:
#iterate over each msno
for i, row in tqdm(trans.iterrows(), total=total_rows):
    msno = row['msno']
    transaction_date = row['transaction_date']
    membership_expire_date = row['membership_expire_date']
    is_cancel = int(row['is_cancel'])
    if(current_msno != msno or i == total_rows - 1):
        if(current_msno != ''):
            
            #iterate over all target months for each msno
            for z, target in enumerate(targets):
                
                #start with latest trasaction date
                for j in range(len(transaction_dates)-1,-1,-1):
                     #store YYYYMM of transaction date
                    l = int(transaction_dates[j]/100)
                     #store YYYYMM of expiry date
                    r = int(membership_expire_dates[j]/100)
                    record_index = j

                    #check if expiry date falls in target month and user hasn't canceled the membership
                    #and if there exists a transaction before (or in) the target month
                    if(r == target and is_cancels[j] == 0 and l < target):
                        #start with assuming that user churned for given target month
                        is_churn = 1
                        expired_date = datetime.datetime.strptime(str(membership_expire_dates[j]), "%Y%m%d")
                        
                        #check for next transaction, after given expiry date
                        for k in range(j+1,len(transaction_dates)):
                            trans_date = datetime.datetime.strptime(str(transaction_dates[k]), "%Y%m%d")
                            expired_date1 = datetime.datetime.strptime(str(membership_expire_dates[k]), "%Y%m%d")
                            is_cancel = is_cancels[k]
                            dif_d = (trans_date-expired_date).days
                            #did user make a transaction within 30 days after expiry date of membership?
                            if(dif_d < 30):
                                if(is_cancel == 0 and expired_date < expired_date1):
                                    is_churn = 0
                            else:
                                break
                        msnos[z].append(current_msno) 
                        is_churns[z].append(is_churn)   
                        break
            
        transaction_dates = []
        membership_expire_dates = []
        is_cancels = []
    transaction_dates.append(transaction_date)
    membership_expire_dates.append(membership_expire_date)
    is_cancels.append(is_cancel)
    
    current_msno = msno

#add data into new csv file (one file per target month)
for i, target in enumerate(targets):
    out = pd.DataFrame()
    out['msno'] = msnos[i]
    out['is_churn'] = is_churns[i]
    out.to_csv('train_v2_final_{}.csv'.format(target), index=False)

100%|████████████████████████████████████████████████████████████████████| 22975416/22975416 [52:47<00:00, 7253.57it/s]
